In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata

# Load datasets
df1 = pd.read_csv("fake_job_postings.csv")
# Dropping columns as per original processing, but keeping text columns for extraction
df1.drop(['benefits', 'company_profile', 'employment_type', 'salary_range',
          'industry', 'department', 'required_experience', 'required_education', 'job_id', 'function',], axis=1, inplace=True)

df2 = pd.read_csv("job_train.csv")
df2 = df2[df2['fraudulent']==1]

# Merge
merged_df = pd.concat([df1, df2], ignore_index=True)

print(f"Total rows: {len(merged_df)}")
merged_df.head()

Total rows: 18336


,title,location,description,requirements,telecommuting,has_company_logo,has_questions,fraudulent
0,Marketing Intern,"US, NY, New York","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0,1,0,0
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0,1,0,0
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever","Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0,1,0,0
3,Account Executive - Washington DC,"US, DC, Washington",THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0,1,0,0
4,Bill Review Manager,"US, FL, Fort Worth",JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0,1,1,0


In [2]:
def remove_accents(text):
    if pd.isna(text):
        return text
    # Normalize to NFD (decompose accented chars), then filter out combining marks
    return ''.join(c for c in unicodedata.normalize('NFD', str(text))
                   if unicodedata.category(c) != 'Mn')

def clean_accents(df, column):
    # Apply remove_accents to the column
    df[column] = df[column].apply(lambda x: remove_accents(x) if not pd.isna(x) else x)

def remove_non_latin(text):
    if pd.isna(text):
        return pd.NA
    cleaned_text = ""
    for char in text:
        if ord(char) > 127:
            continue
        cleaned_text += char
    return cleaned_text

# Clean text columns
text_cols = ['location', 'description', 'requirements']
for col in text_cols:
    clean_accents(merged_df, col)
    merged_df[col] = merged_df[col].apply(remove_non_latin)

print("Text columns cleaned.")

Text columns cleaned.


In [3]:
# 1. Build a set of known locations (Cities) from the existing 'location' column
known_cities = set()

# Regex to capture the city part (assuming "Country, State, City" format)
# We take the part after the second comma, or if only one comma, maybe the second part?
# Let's look at the data: "US, NY, New York", "NZ, , Auckland"
# It seems consistent: 3 parts separated by commas.

def extract_city_from_location(loc_str):
    if pd.isna(loc_str):
        return None
    parts = [p.strip() for p in loc_str.split(',')]
    if len(parts) >= 3:
        city = parts[2]
        if city: # if not empty string
            return city
    return None

# Extract cities from non-missing locations
valid_locations = merged_df['location'].dropna()
for loc in valid_locations:
    city = extract_city_from_location(loc)
    if city:
        known_cities.add(city)

print(f"Found {len(known_cities)} unique cities from existing data.")
# print(list(known_cities)[:10])

# 2. Identify rows with missing location
missing_loc_mask = merged_df['location'].isna()
print(f"Rows with missing location: {missing_loc_mask.sum()}")

# 3. Search for known cities in 'description' and 'requirements'
# We'll combine description and requirements for search
merged_df['combined_text'] = merged_df['description'].fillna('') + " " + merged_df['requirements'].fillna('')

extracted_locations = []

# Compile regex patterns for cities to speed up search? 
# Or just iterate? Iterating 2000+ cities over 18000 rows might be slow (36M operations).
# Better to use regex with OR operator for batches of cities, or Aho-Corasick if available (not standard lib).
# Let's try a simple iteration first, or maybe regex for whole words.
# \b(City1|City2|...)\b
# If the list is too long, regex might fail. 2000 cities is okay-ish.

# Let's filter known_cities to remove very short common words if any (e.g. "A", "Of")
# Although cities are usually distinct.
filtered_cities = [c for c in known_cities if len(c) > 2] # Filter out very short abbreviations just in case
print(f"Using {len(filtered_cities)} cities for search.")

# Create a regex pattern
# Escape cities to handle special chars
import re
escaped_cities = [re.escape(c) for c in filtered_cities]
# Sort by length descending to match longest city names first (e.g. "New York" before "York")
escaped_cities.sort(key=len, reverse=True)

# We can't put all 2000 in one regex group, it might exceed limits.
# But let's try.
city_pattern = r'\b(' + '|'.join(escaped_cities) + r')\b'

def find_location(text):
    if not text:
        return None
    match = re.search(city_pattern, text, re.IGNORECASE) # Case insensitive search? Locations in text might be capitalized, but maybe not.
    if match:
        return match.group(0)
    return None

# Apply only to rows with missing location
# Note: This might take a while if the regex is huge.
# Let's test on a sample first or just run it.

# Optimization: Only run on missing rows
missing_indices = merged_df[missing_loc_mask].index

print("Starting extraction...")
# We can use apply, but let's be careful with the huge regex.
# If it's too slow, we might need another approach.
# Let's try on the subset.

extracted_series = merged_df.loc[missing_indices, 'combined_text'].apply(find_location)

# 4. Create a new column
merged_df['extracted_location'] = merged_df['location'] # Start with original
merged_df.loc[missing_indices, 'extracted_location'] = extracted_series

print(f"Extracted locations for {extracted_series.notna().sum()} rows out of {len(missing_indices)} missing.")

merged_df[['location', 'extracted_location']].head()

Found 2191 unique cities from existing data.
Rows with missing location: 354
Using 2181 cities for search.
Starting extraction...
Extracted locations for 259 rows out of 354 missing.


,location,extracted_location
0,"US, NY, New York","US, NY, New York"
1,"NZ, , Auckland","NZ, , Auckland"
2,"US, IA, Wever","US, IA, Wever"
3,"US, DC, Washington","US, DC, Washington"
4,"US, FL, Fort Worth","US, FL, Fort Worth"


In [4]:
# Save the result
output_file = "fake_job_postings_extracted_loc.csv"
merged_df.to_csv(output_file, index=False)
print(f"Saved to {output_file}")

# Display some examples where location was missing but extracted
extracted_examples = merged_df[merged_df['location'].isna() & merged_df['extracted_location'].notna()]
print("Examples of extracted locations:")
print(extracted_examples[['combined_text', 'extracted_location']].head())

Saved to fake_job_postings_extracted_loc.csv
Examples of extracted locations:
                                         combined_text extracted_location
144  The group has raised a fund for the purchase o...           purchase
204  Who were looking forMaker MentalityAre you foc...                new
234  This unique, field-based, full-time program br...               city
325  Following our global expansion we are seeking ...                all
349  SummaryImmediate requirement for an advanced t...           multiple


In [6]:
merged_df['extracted_location']

0            US, NY, New York
1              NZ, , Auckland
2               US, IA, Wever
3          US, DC, Washington
4          US, FL, Fort Worth
                 ...         
18331           US, , chicago
18332    US, NY, NEWYORK CITY
18333       US, NY, Rochester
18334       US, CA, San Mateo
18335       US, CA, Santa Ana
Name: extracted_location, Length: 18336, dtype: object